In [1]:
from torchvision.models.detection import SSDLite320_MobileNet_V3_Large_Weights, ssdlite320_mobilenet_v3_large
# from ssdlite.ssdlite import ssdlite320_mobilenet_v3_large, SSDLite320_MobileNet_V3_Large_Weights

In [2]:
model = ssdlite320_mobilenet_v3_large(weights=SSDLite320_MobileNet_V3_Large_Weights.DEFAULT)

In [3]:
model.eval()
model.to('cpu')

SSD(
  (backbone): SSDLiteFeatureExtractorMobileNet(
    (features): Sequential(
      (0): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (2): Hardswish()
        )
        (1): InvertedResidual(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
              (1): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
              (2): ReLU(inplace=True)
            )
            (1): Conv2dNormActivation(
              (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (1): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
            )
          )
        )
        (2): Invert

In [4]:
from torch.utils.mobile_optimizer import optimize_for_mobile
import torch

In [ ]:
example = torch.rand(1, 3, 320, 320z)

In [5]:
from PIL import Image
import numpy as np
import torch

image = Image.open("/home/fano/Desktop/CSCI 494/final_project/Object-detection-using-MobileNet/data/images/000039.jpg")
img = torch.from_numpy(np.array(image).astype(np.float32).transpose(2, 0, 1)).unsqueeze(0) / 255

In [6]:
torch.save(model.state_dict(), "orig_model.pt")


model.qconfig = torch.ao.quantization.get_default_qconfig('x86')
model_prepared = torch.ao.quantization.prepare(model)

# input_fp32 = torch.randn(4, 1, 4, 4)
model_prepared(img)
model_int8 = torch.ao.quantization.convert(model_prepared)

# also quant and dequat should be added to SSD forward method

torchscript_model = torch.jit.script(model_int8)
optimizered_ts_model = optimize_for_mobile(torchscript_model)
torch.jit.save(optimizered_ts_model, "model.pt")

/home/fano/miniconda3/envs/sergek/lib/python3.10/site-packages/torch/ao/quantization/observer.py:221: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


In [7]:
with torch.no_grad():
    outputs = model_int8(img)

NotImplementedError: Could not run 'aten::add.out' with arguments from the 'QuantizedCPU' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'aten::add.out' is only available for these backends: [CPU, CUDA, Meta, MkldnnCPU, SparseCPU, SparseCUDA, SparseMeta, SparseCsrCPU, SparseCsrCUDA, SparseCsrMeta, BackendSelect, Python, FuncTorchDynamicLayerBackMode, Functionalize, Named, Conjugate, Negative, ZeroTensor, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradHIP, AutogradXLA, AutogradMPS, AutogradIPU, AutogradXPU, AutogradHPU, AutogradVE, AutogradLazy, AutogradMTIA, AutogradPrivateUse1, AutogradPrivateUse2, AutogradPrivateUse3, AutogradMeta, AutogradNestedTensor, Tracer, AutocastCPU, AutocastXPU, AutocastCUDA, FuncTorchBatched, BatchedNestedTens

In [ ]:
coco_names = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
    'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table',
    'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book',
    'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

In [ ]:
# get score for all the predicted objects
pred_scores = outputs[0]['scores'].detach().cpu().numpy()
# get all the predicted bounding boxes
pred_bboxes = outputs[0]['boxes'].detach().cpu().numpy()
# get boxes above the threshold score
boxes = pred_bboxes[pred_scores >= 0.8].astype(np.int32)
labels = outputs[0]['labels'][:len(boxes)]
# get all the predicited class names
pred_classes = [coco_names[i] for i in labels.cpu().numpy()]

In [ ]:
boxes, pred_classes, labels

In [ ]:
import cv2
import os

In [ ]:
def draw_boxes(boxes, classes, image):
    """
    Draws the bounding box around a detected object.
    """
    image = cv2.cvtColor(np.asarray(image), cv2.COLOR_BGR2RGB)
    for i, box in enumerate(boxes):
        color = (255, 255, 0)
        cv2.rectangle(
            image,
            (int(box[0]), int(box[1])),
            (int(box[2]), int(box[3])),
            color, 2
        )
        cv2.putText(image, classes[i], (int(box[0]), int(box[1]-5)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2, 
                    lineType=cv2.LINE_AA)
    return image

In [ ]:
result_image = draw_boxes(boxes, pred_classes, image)

In [ ]:
cv2.imwrite(f"outputs.jpg", result_image)

In [ ]:
# model.qconfig = torch.ao.quantization.get_default_qconfig('x86')
# model_fp32_fused = torch.ao.quantization.fuse_modules(model, [['conv', 'batchnorm' ,'relu']])
# model_fp32_prepared = torch.ao.quantization.prepare(model_fp32_fused)

# calibrate with real dataset ??

model_int8 = torch.ao.quantization.quantize_dynamic(
    model,  # the original model
    {torch.nn.Linear, torch.nn.Conv2d, torch.nn.BatchNorm2d},  # a set of layers to dynamically quantize
    dtype=torch.qint8)  # the target dtype for quantized weights


In [ ]:
with torch.no_grad():
    outputs = model_int8(img)

In [ ]:
# get score for all the predicted objects
pred_scores = outputs[0]['scores'].detach().cpu().numpy()
# get all the predicted bounding boxes
pred_bboxes = outputs[0]['boxes'].detach().cpu().numpy()
# get boxes above the threshold score
boxes = pred_bboxes[pred_scores >= 0.8].astype(np.int32)
labels = outputs[0]['labels'][:len(boxes)]
# get all the predicited class names
pred_classes = [coco_names[i] for i in labels.cpu().numpy()]

In [ ]:
result_image = draw_boxes(boxes, pred_classes, image)

In [ ]:
cv2.imwrite(f"outputs1.jpg", result_image)

In [ ]:
def print_model_size(mdl):
    torch.save(mdl.state_dict(), "tmp.pt")
    print("%.2f MB" %(os.path.getsize("tmp.pt")/1e6))
    os.remove('tmp.pt')

print_model_size(model_int8)
print_model_size(model)